<a href="https://colab.research.google.com/github/kaitehtzeng/LLM_MT_DF/blob/main/src/wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00


In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
pwd

'/content'

In [ ]:
from unsloth import FastLanguageModel
import logging
import os
from contextlib import nullcontext

TRL_USE_RICH = os.environ.get("TRL_USE_RICH", False)

from trl.commands.cli_utils import init_zero_verbose, SFTScriptArguments, TrlParser
from prompt_temple import formatting_prompts_func,end_of_prompt
if TRL_USE_RICH:
    init_zero_verbose()
    FORMAT = "%(message)s"

from rich.console import Console
from rich.logging import RichHandler

import torch
from datasets import load_dataset

from tqdm.rich import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import random

from trl import (
    ModelConfig,
    RichProgressCallback,
    SFTConfig,
    SFTTrainer,
    get_peft_config,
    get_quantization_config,
    get_kbit_device_map,
    DataCollatorForCompletionOnlyLM,
)
from torchtext.data.metrics import bleu_score
import wandb

wandb.init(project="slightly-tuned llma3 model")
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = fp16,
        load_in_4bit = True
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

eval_dataset = load_dataset("json",data_files=eval_file_path)['train']
eval_target = [item['tgt'] for item in eval_dataset]
eval_dataset = eval_dataset.map(formating_prompts_func_eval,5)

def generate_batch(text,model,tokenizer):
    input_id = tokenizer(text,return_tensors= pt, padding =True).to(model.device)
    output = model.generate(
        **input,
        max_new_tokens=256,
        num_beams=5,
        early_stopping=True,
        do_sample=False)
    mount = []
    for input,output_id in zip(input_id['input_ids'],output):
        mount.append(output_id[input.size(0):])
    tok = tokenizer.batch_decode(mount, skip_special_tokens=True)
    return tok


def generate_example(dataset,batch_size,model,tokenizer):
    ans = []
    for i in range(0,len(dataset),5):
        batch = generate_batch(data[i:min(len(dataset),i+batch_size)],model,tokenizer)
        ans.extend(batch)
    return ans

result = generate_example(eval_dataset,5,model,tokenizer)

result_corpus = [i.split() for i in result]
target_corpus = [[i.split()] for i in eval_target]
bleu = bleu_score(result_corpus)
wandb.log({'bleu':bleu})

for i in range(len(eval_dataset)):
    wandb.log({'text':eval_dataset[i],'true':eval_target[i],'prediction':result[i]})

wandb.finish()

